In [5]:
import pickle
import numpy as np
import scipy.spatial
from sklearn.preprocessing import StandardScaler
import torch
import os
import matplotlib.pyplot as plt
import imp

vis_data_by_label = imp.load_source('vis_data_by_label', '../visualisation/data_vis.py')




In [7]:
ims = pickle.load(open('../data/images_list.pickle', 'rb'))
labels = pickle.load(open('../data/lettersdf.pickle', 'rb'))

layers = [0, -7, -14]
all_data = {'gram' : {k:None for k in layers},
            'raw' : {k:None for k in layers}}

In [9]:
for case in ['gram', 'raw']:
    for l in layers:

        
        if case == 'gram':
            file = './Reps/VGG16_Initials_Gram{}default.pickle'.format(l)
        else:
            file = './Reps/VGG16_Initials_{}untrained_binary.pickle'.format(l)

        data = pickle.load(open(file, 'rb'))
        if type(data)==list:
            data = torch.stack([i.flatten() for i in data])
        print(case, l, data.shape)
        
        all_data[case][l] = data.numpy()

FileNotFoundError: [Errno 2] No such file or directory: './Reps/VGG16_Initials_Gram0default.pickle'

In [1]:
# get list of all the files in a cluster

cluster = pickle.load(open('../data/mauresque.pickle', 'rb'))


NameError: name 'pickle' is not defined

In [6]:
c_labels = ([None] * len(ims)).append([None])
labels["cluster_labels"] = c_labels


for row in labels.iterrows():
    #print(row[0])
    if row[1]["initial"] in cluster:
        labels.loc[row[0], "cluster_labels"] = "cluster_arabesque"

labels[labels["cluster_labels"] == "cluster_arabesque"]
labels['cluster_labels']


0       None
1       None
2       None
3       None
4       None
        ... 
5858    None
5859    None
5860    None
5861    None
5862    None
Name: cluster_labels, Length: 5863, dtype: object

In [7]:
c_labels = [list(labels[labels.initial == im].cluster_labels)[0] for im in ims]

c_labels = np.array(c_labels)

len(c_labels)

5862

In [8]:
# compute invariance score of cluster 

idxs = np.array(list(range(5862)))
np.random.shuffle(idxs)
rand_idxs = idxs[:2000]
standardize = 'std'

for metric in ['euclidean', 'cosine']:
    print(metric)
    dist_data = {}
    for case in ['gram', 'raw']:

        dist_data[case] = {l: {} for l in layers}


        for l in layers:
            data = all_data[case][l]
            print(data.shape)
          
            if standardize == 'std':
                scaler = StandardScaler()
                scaler.fit(data)
                data = scaler.transform(data)
            
            dists_all = scipy.spatial.distance.pdist(data[rand_idxs,:], metric = metric) 
                
            for cluster_label in list(set(c_labels)):

                if cluster_label != None:

                    print(cluster_label, len(np.where(c_labels == 'cluster_arabesque')[0]))
                                
                    data_letter = data[(np.where(c_labels == 'cluster_arabesque')[0])]                        
                    dists_letter = scipy.spatial.distance.pdist(data_letter, metric = metric)
                    dist_data[case][l] = (dists_letter.mean(), dists_all.mean(), len(np.where(c_labels == 'cluster_arabesque')[0]))
              

    pickle.dump(dist_data, open('data_cluster_default{}_{}_mauresque.p'.format(metric, standardize), 'wb'))

euclidean
(5862, 262144)
cluster_arabesque 180
(5862, 262144)
cluster_arabesque 180
(5862, 262144)
cluster_arabesque 180
(5862, 25088)
cluster_arabesque 180
(5862, 100352)
cluster_arabesque 180
(5862, 200704)
cluster_arabesque 180
